In [1]:
import numpy as np
# from pymoo.core.problem import Problem
from scipy.interpolate import griddata
import pandas as pd

from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.core.problem import ElementwiseProblem
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.repair.rounding import RoundingRepair
from pymoo.operators.sampling.rnd import IntegerRandomSampling
from pymoo.optimize import minimize
from pymoo.termination.default import DefaultSingleObjectiveTermination

In [2]:
jpeg_cha = pd.read_csv("../../stable_tests/measurements/jpeg_snr_cha/characteristic.csv")

jpeg_cha = jpeg_cha[jpeg_cha["sparsity"]>0]
jpeg_cha = jpeg_cha[jpeg_cha["datasize_est"]>0]
tensor_size = 128*26*26 *4
jpeg_cha["ratio"] = tensor_size/jpeg_cha["datasize_est"]

In [3]:
pruning = [0.05, 0.1, 0.15, 0.2, 0.25,0.3,0.35]
quality = [60,70,80,90,100]

sample_points=[]
cmp_sample_values = []
snr_sample_values = []

cha_df_group =jpeg_cha.groupby("pruning_thresh")
for p in pruning:
    cha_df = cha_df_group.get_group(p)
    cha_quality_df = cha_df.groupby("quality")
    for q in quality:
        cha_plot_df= cha_quality_df.get_group(q)
        sample_points.append([q,p])
        cmp_sample_values.append(cha_plot_df["ratio"].mean())
        snr_sample_values.append(cha_plot_df["reconstruct_snr"].mean())

sample_points = np.array(sample_points)
cmp_sample_values = np.array(cmp_sample_values)
snr_sample_values = np.array(snr_sample_values)

In [4]:
# Interpolate the constraint function
def interpolated_constraint_cmp(xy):
    return griddata(sample_points, cmp_sample_values, (xy[0], xy[1]/100), method='cubic')

def interpolated_constraint_snr(xy):
    return griddata(sample_points, snr_sample_values, (xy[0], xy[1]/100), method='cubic')


class CustomProblem(ElementwiseProblem):

    def __init__(self,snr,cmp):
        self.snr = snr
        self.cmp = cmp
        super().__init__(n_var=2, n_obj=1, n_ieq_constr=2, xl=[60,5], xu=[100,35],vtype=int)

    def _evaluate(self, x, out, *args, **kwargs):
        out["F"] = x[1]/35-x[0]/100

        constraint_1 = self.cmp-interpolated_constraint_cmp(x)
        constraint_2 = self.snr-interpolated_constraint_snr(x)

        out["G"] = np.array([
            constraint_1,constraint_2
        ])
        penalty = 1000  # Large penalty factor
        if constraint_1 > 0:  # If constraint is violated
            out["F"] += penalty * constraint_1
        if constraint_1 > 0:  # If constraint is violated
            out["F"] += penalty * constraint_2

In [13]:
# Initialize the problem
problem = CustomProblem(snr=5,cmp=100)

# Select the algorithm (Genetic Algorithm in this case)
algorithm = GA(pop_size=25,
            sampling=IntegerRandomSampling(),
            crossover=SBX(prob=1.0, eta=3.0, vtype=float, repair=RoundingRepair()),
            mutation=PM(prob=1.0, eta=3.0, vtype=float, repair=RoundingRepair()),
            eliminate_duplicates=True,
            )

termination = DefaultSingleObjectiveTermination(
    xtol=1e-8,
    cvtol=1e-6,
    ftol=1e-6,
    period=20,
    n_max_gen=10,
    n_max_evals=10000
)


# Perform the optimization
result = minimize(problem, algorithm, termination=termination,verbose=True)

# # Get and display the results
# optimal_x, optimal_y = result.X
# optimal_objective_value = -result.F[0]

print("Best solution found: \nX = %s\nF = %s" % (result.X, result.F))

n_gen  |  n_eval  |     cv_min    |     cv_avg    |     f_avg     |     f_min    
     1 |       25 |  0.2386314213 |  2.857361E+01 |             - |             -
     2 |       50 |  0.000000E+00 |  2.4903557239 | -2.685714E-01 | -2.685714E-01
     3 |       75 |  0.000000E+00 |  1.4763792654 | -2.557143E-01 | -2.685714E-01
     4 |      100 |  0.000000E+00 |  0.9693057370 | -2.557143E-01 | -2.685714E-01
     5 |      125 |  0.000000E+00 |  0.8242040185 | -2.566667E-01 | -2.685714E-01
     6 |      150 |  0.000000E+00 |  0.7439827628 | -2.566667E-01 | -2.685714E-01
     7 |      175 |  0.000000E+00 |  0.5605342936 | -2.430952E-01 | -2.685714E-01
     8 |      200 |  0.000000E+00 |  0.4827028318 | -2.442857E-01 | -2.685714E-01
     9 |      225 |  0.000000E+00 |  0.4272549479 | -2.421429E-01 | -2.685714E-01
    10 |      250 |  0.000000E+00 |  0.3415962797 | -2.442857E-01 | -2.685714E-01
Best solution found: 
X = [84 20]
F = [-0.26857143]


In [14]:
interpolated_constraint_cmp(result.X)

array(100.38052828)

In [15]:
interpolated_constraint_snr(result.X)

array(5.66590749)